<a href="https://colab.research.google.com/github/Winzen/mides-rascunho/blob/main/code/scraping/ro/%5Bro%5Dextrair.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Verificar IP

In [ ]:
import requests

my_country = requests.get("https://api.myip.com/")
my_country.json()

{'ip': '34.73.169.177', 'country': 'United States', 'cc': 'US'}

# Importação

In [ ]:
import requests
from bs4 import BeautifulSoup
import concurrent.futures
from urllib3 import PoolManager
import pandas as pd
import os
from typing import Type
import glob
import shutil


def get_dados_from_municipio_ano_entidade(id_municipio: str|int,
                                          id_entidade:
                                          str|int, ano: str|int) -> list:
  # Define the URL you want to send a POST request to
  url = 'https://transparencia.tce.ro.gov.br/transparenciatce/Remessa/Pesquisar'

  # Define the data you want to send in the POST request
  data = {
  "tipo":	"contabil",
  "esfera":	"municipal",
  "municipios":	id_municipio,
  "entidades":	id_entidade,
  "exercicios":	ano,
  "mes":	""}

  # Make the POST request
  response = http.request('POST', url, fields=data)

  soup = BeautifulSoup(response.data, 'html.parser')
  base_url = "https://transparencia.tce.ro.gov.br"
  links_zips = [base_url + str(zip.get("href")) for zip in soup.select("td a")]
  rows = [[ano, id_municipio, id_entidade, link] for link in links_zips]

  return rows

def get_entidades_from_municipio(municipio: str, id_municipio: str|int) -> list:

  url = f"https://transparencia.tce.ro.gov.br/transparenciatce/Remessa/GetEntidades?esferaCod=2&municipio={id_municipio}"
  # response = requests.get(url)
  response = http.request('GET', url)

  json_response = response.json()
  entidades = [[municipio, id_municipio, entidade.get("Text"), entidade.get("Value")] for entidade in json_response]
  return entidades

def get_municipios() -> list:

  url = 'https://transparencia.tce.ro.gov.br/transparenciatce/Remessa/Pesquisar'
  # Define the data you want to send in the POST request
  data = {
  "tipo":	"contabil",
  "esfera":	"municipal",
  "municipios":	2,
  "entidades":	2,
  "exercicios":	2019,
  "mes":	""}

  # Make the POST request
  response = requests.post(url, data=data)

  soup = BeautifulSoup(response.text, 'html.parser')
  municipios = [text_select.text for text_select in soup.select("select#municipios option")[1::]] # Excluir primeiro elemento desnecessario e forma lista com municipios
  return municipios


def download_large_file(row: Type):

  zip_name = get_name_zip_from_link(row.link)

  path_directory = os.path.join(os.getcwd(), "input",
                          str(row.ano),
                          str(row.id_municipio),
                          row.id_entidade)

  file_name = os.path.join(path_directory, zip_name)

  os.makedirs(path_directory, exist_ok=True)

  # Make a GET request with stream=True to download the file in chunks
  if not  os.path.exists(file_name):
    with http.request('GET', row.link, preload_content=False, decode_content=False) as response:
        # Check if the request was successful (status code 200)
        if response.status == 200:
            # Open a local file for writing in binary mode
            with open(file_name, 'wb') as file:
                # Download the file in chunks and save to the local disk
                for chunk in response.stream(8192): # Adjust chunk size as needed
                    file.write(chunk)
            # print(f"Download complete. File saved as {zip_name}")
        else:
            print(f"Error: Unable to download file. Status Code: {response.status}\nLink:{row.link}")

def get_name_zip_from_link(link: str) -> str:
  return link.split("=")[1].split("&")[0]


def send_folder_drive(path_drive: str, path_input: str) -> None:

  if not len(os.listdir(path_input)) > 0:
    raise Exception("Essa Pasta vazia")
    return None

  shutil.make_archive(path_drive,
                      'zip',
                      path_input)



# Pegar Municipios Id e Entidades

In [ ]:
municipios = get_municipios()
http = PoolManager(maxsize=10)
with concurrent.futures.ThreadPoolExecutor() as executor:
  entidades = [row
               for entidades_municipio in
               executor.map(
                   get_entidades_from_municipio,
                   municipios,
                   list(range(1, len(municipios) + 1)))
               for row in entidades_municipio
               ]

df_entidaes = pd.DataFrame(entidades, columns=["municipio", "id_municipio", "entidade", "id_entidade"])
df_entidaes.to_csv("/content/utils/base_info_entidades.csv", index=False)

# Pegar links dos ZIPs

In [ ]:
rows_link_by_entidade = []
http = PoolManager(maxsize=10)

for pesquisa_row in df_entidaes.itertuples():

  with concurrent.futures.ThreadPoolExecutor() as executor:
    rows_link_by_entidade += [row
                for zips in
                executor.map(
                    lambda ano: get_dados_from_municipio_ano_entidade(pesquisa_row.id_municipio, pesquisa_row.id_entidade, ano),
                    list(range(2019,2025))
                    )
                for row in zips
               ]

df_links_zips = pd.DataFrame(rows_link_by_entidade, columns=["ano", "id_municipio", "id_entidade", "link"])
df_links_zips.to_csv("/content/utils/link_zips_ro.csv", index=False)

# Download dos Zips

In [ ]:
with concurrent.futures.ThreadPoolExecutor() as executor:
  executor.map(
              download_large_file,
              df_links_zips.itertuples()
              )

Error: Unable to download file. Status Code: 500
Link:https://transparencia.tce.ro.gov.br/transparenciatce/Remessa/Download?name=Sigap_201901_41_04394805000118_CO_SMVDDM-167.zip&fullPath=%5C%5Ctcero.local%5Csistemas%5CFILES%5CSigapMunicipal%5C2019%5CSigap_201901_41_04394805000118_CO_SMVDDM-167.zip&esfera=municipal
Error: Unable to download file. Status Code: 500
Link:https://transparencia.tce.ro.gov.br/transparenciatce/Remessa/Download?name=Sigap_201901_67_04390985000160.rar&fullPath=%5C%5Ctcero.local%5Csistemas%5CFILES%5CSigapMunicipal%5C2019%5CSigap_201901_67_04390985000160.rar&esfera=municipal


# Manda para o Drive

In [ ]:
path_drive_output = f"/content/drive/MyDrive/DataBase/world_bank/ro/empenhos/empenho_ro"

send_folder_drive(path_drive_output,
                "/content/input/")

In [ ]:
path_drive_utils = f"/content/drive/MyDrive/DataBase/world_bank/ro/utils/utils"


send_folder_drive(path_drive_output,
                "/content/utils")